In [ ]:
'''
This is the test code for visualizing poisoned training on CIFAR10, MNIST, using dataset class of torchvision.datasets.DatasetFolder torchvision.datasets.CIFAR10 torchvision.datasets.MNIST.
Attack method is Refool.
'''

import os
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, dataloader
import torchvision
from torchvision import transforms
from torchvision.datasets import CIFAR10, MNIST, DatasetFolder
from torchvision.transforms import Compose, ToTensor, PILToTensor, RandomHorizontalFlip
from torch.utils.data import DataLoader
import core
import matplotlib.pyplot as plt

In [ ]:
global_seed = 666
deterministic = True
torch.manual_seed(global_seed)
reflection_data_dir = "/data/ganguanhao/datasets/VOCdevkit/VOC2012/JPEGImages/"
def read_image(img_path, type=None):
    img = cv2.imread(img_path)
    if type is None:        
        return img
    elif isinstance(type,str) and type.upper() == "RGB":
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    elif isinstance(type,str) and type.upper() == "GRAY":
        return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        raise NotImplementedError
    
reflection_image_path = os.listdir(reflection_data_dir)
reflection_images = [read_image(os.path.join(reflection_data_dir,img_path)) for img_path in reflection_image_path[:200]]

def show_img(x):
    img = cv2.cvtColor(x.permute(1,2,0).numpy(),cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

In [ ]:
transform_train = Compose([
    transforms.ToPILImage(),
    # transforms.Resize((32,32)),
    RandomHorizontalFlip(),
    ToTensor(),
])
transform_test = Compose([
    transforms.ToPILImage(),
    # transforms.Resize((32,32)),
    ToTensor(),
])

trainset = DatasetFolder(
    root='/data/ganguanhao/datasets/cifar10-folder/trainset',
    loader=cv2.imread,
    extensions=('png',),
    transform=transform_train,
    target_transform=None,
    is_valid_file=None)

testset = DatasetFolder(
    root='/data/ganguanhao/datasets/cifar10-folder/testset',
    loader=cv2.imread,
    extensions=('png',),
    transform=transform_train,
    target_transform=None,
    is_valid_file=None)

refool= core.Refool(
    train_dataset=trainset,
    test_dataset=testset,
    model=core.models.ResNet(18),
    loss=nn.CrossEntropyLoss(),
    y_target=1,
    poisoned_rate=0.05,
    poisoned_transform_train_index=0,
    poisoned_transform_test_index=0,
    poisoned_target_transform_index=0,
    schedule=None,
    seed=global_seed,
    deterministic=deterministic,
    reflection_candidates = reflection_images,
)


poisoned_train_dataset, poisoned_test_dataset = refool.get_poisoned_dataset()
print("============train_images============")
for i in range(10):
    show_img(poisoned_train_dataset[i][0])
print("============test_images============")
for i in range(10):
    show_img(poisoned_test_dataset[i][0])


In [ ]:
transform_train = Compose([
    RandomHorizontalFlip(),
    ToTensor(),
])
transform_test = Compose([
    ToTensor(),
])
trainset = CIFAR10(
    root='/data/ganguanhao/datasets',
    transform=transform_train,
    target_transform=None,
    train=True,
    download=True)
testset = CIFAR10(
    root='/data/ganguanhao/datasets',
    transform=transform_test,
    target_transform=None,
    train=False,
    download=True)

refool= core.Refool(
    train_dataset=trainset,
    test_dataset=testset,
    model=core.models.ResNet(18),
    loss=nn.CrossEntropyLoss(),
    y_target=1,
    poisoned_rate=0.05,
    poisoned_transform_train_index=0,
    poisoned_transform_test_index=0,
    poisoned_target_transform_index=0,
    schedule=None,
    seed=global_seed,
    deterministic=deterministic,
    reflection_candidates = reflection_images,
)


poisoned_train_dataset, poisoned_test_dataset = refool.get_poisoned_dataset()
print("============train_images============")
for i in range(10):
    show_img(poisoned_train_dataset[i][0])
print("============test_images============")
for i in range(10):
    show_img(poisoned_test_dataset[i][0])

In [ ]:
transform_train = Compose([
    RandomHorizontalFlip(),
    ToTensor(),
])
transform_test = Compose([
    ToTensor(),
])
trainset = MNIST(
    root='/data/ganguanhao/datasets',
    transform=transform_train,
    target_transform=None,
    train=True,
    download=True)
testset = MNIST(
    root='/data/ganguanhao/datasets',
    transform=transform_test,
    target_transform=None,
    train=False,
    download=True)

loader = DataLoader(trainset,)

refool= core.Refool(
    train_dataset=trainset,
    test_dataset=testset,
    model=core.models.BaselineMNISTNetwork(),
    loss=nn.CrossEntropyLoss(),
    y_target=1,
    poisoned_rate=0.05,
    poisoned_transform_train_index=0,
    poisoned_transform_test_index=0,
    poisoned_target_transform_index=0,
    schedule=None,
    seed=global_seed,
    deterministic=deterministic,
    reflection_candidates = reflection_images,
)

poisoned_train_dataset, poisoned_test_dataset = refool.get_poisoned_dataset()
print("============train_images============")
for i in range(10):
    show_img(poisoned_train_dataset[i][0])
print("============test_images============")
for i in range(10):
    show_img(poisoned_test_dataset[i][0])